In [1]:
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
from tkinter import filedialog

import matplotlib
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import numpy as np
import imageio
import re

plt.rcParams['image.cmap'] = 'gray'

In [2]:
def rgb_yiq(img):
    yiq = np.zeros(img.shape)
    yiq[:,:,0] = 0.229*img[:,:,0]+0.587*img[:,:,1]+0.114*img[:,:,2]
    yiq[:,:,1] = 0.595716*img[:,:,0]-0.274453*img[:,:,1]-0.321263*img[:,:,2]
    yiq[:,:,2] = 0.211456*img[:,:,0]-0.522591*img[:,:,1]+0.311135*img[:,:,2]
    return yiq

def yiq_rgb(img):
    rgb = np.zeros(img.shape)
    rgb[:,:,0] = np.clip(img[:,:,0]+0.9663*img[:,:,1]+0.6210*img[:,:,2],0,1)
    rgb[:,:,1] = np.clip(img[:,:,0]-0.2721*img[:,:,1]-0.6474*img[:,:,2],0,1)
    rgb[:,:,2] = np.clip(img[:,:,0]-1.1070*img[:,:,1]+1.7046*img[:,:,2],0,1)
    return rgb

def es_decimal(cadena):
    patron = r'^[0-9]+(\.[0-9])$'
    regex = re.compile(patron)
    return bool(regex.search(cadena))

#Funciones de trabajo practico 4
#La siguiente funcion procesa una imagen con un kernel (donde se muliplican y luego se suman y ese resultado es el nuevo valor del pixel que se estaba procesando)
def process(img, kernel):
    res=np.zeros(img.shape)
    if (kernel.shape[0])==3:               #Se procesa con un Kernel de 3x3
        aux = np.zeros(kernel.shape)
        res[:,0] = img[:,0]
        res[0,:] = img[0,:]
        res[:,res.shape[1]-1] = img[:,res.shape[1]-1]
        res[res.shape[0]-1,:] = img[res.shape[0]-1,:]
        for i in range(1,res.shape[0]-1):
            for j in range(1,res.shape[1]-1):
                for ik in range(3):
                    for jk in range(3):
                        aux[ik,jk] = kernel[ik,jk]*img[i-1+ik,j-1+jk]
                res[i,j] = np.clip(np.sum(aux),0.,1.)
    elif (kernel.shape[0])==5:              #Se procesa con un kernel de 5x5
        aux = np.zeros(kernel.shape)
        res[:,0] = img[:,0]
        res[:,1] = img[:,1]
        res[0,:] = img[0,:]
        res[1,:] = img[1,:]
        res[:,res.shape[1]-1] = img[:,res.shape[1]-1]
        res[:,res.shape[1]-2] = img[:,res.shape[1]-2]
        res[res.shape[0]-1,:] = img[res.shape[0]-1,:]
        res[res.shape[0]-2,:] = img[res.shape[0]-2,:]
        for i in range(2,res.shape[0]-2):
            for j in range(2,res.shape[1]-2):
                for ik in range(5):
                    for jk in range(5):
                        aux[ik,jk] = kernel[ik,jk]*img[i-2+ik,j-2+jk]
                res[i,j] = np.sum(aux)
    else:                                #Se procesa con un kernel de 7x7
        aux = np.zeros(kernel.shape)
        res[:,0] = img[:,0]
        res[:,1] = img[:,1]
        res[:,2] = img[:,2]
        res[0,:] = img[0,:]
        res[1,:] = img[1,:]
        res[2,:] = img[2,:]
        res[:,res.shape[1]-1] = img[:,res.shape[1]-1]
        res[:,res.shape[1]-2] = img[:,res.shape[1]-2]
        res[:,res.shape[1]-3] = img[:,res.shape[1]-3]
        res[res.shape[0]-1,:] = img[res.shape[0]-1,:]
        res[res.shape[0]-2,:] = img[res.shape[0]-2,:]
        res[res.shape[0]-3,:] = img[res.shape[0]-3,:]
        for i in range(3,res.shape[0]-3):
            for j in range(3,res.shape[1]-3):
                for ik in range(7):
                    for jk in range(7):
                        aux[ik,jk] = kernel[ik,jk]*img[i-3+ik,j-3+jk]
                res[i,j] = np.sum(aux)
    return res
                            
#Definicion de la funcion plano
def plano(img):
    vec= [1,1,1]                   #Se define el vector que es igual tanto en la primera fila como columna del kernel
    [x,y] = np.meshgrid(vec,vec)
    kernel = (x*y)/np.sum(x*y)       #Se crea el kernel(Se crea la matriz y a cada valor se lo divide por la sumatoria de todos los valores en la matriz)
    if len(img.shape)==3:               
        return process(img[:,:,0],kernel)    #Se llama a la funcion que procesa la imagen con el kernel
    else:
        return process(img,kernel)
                               
#Definicion de la funcion Bartlett de 3x3
def bart_3x3(img):
    vec= [1,2,1]
    [x,y] = np.meshgrid(vec,vec)
    kernel = (x*y)/np.sum(x*y)
    if len(img.shape)==3:
        return process(img[:,:,0],kernel)
    else:
        return process(img,kernel)

#Definicion de la funcion Bartlett de 5x5
def bart_5x5(img):
    vec= [1,2,3,2,1]
    [x,y] = np.meshgrid(vec,vec)
    kernel = (x*y)/np.sum(x*y)
    if len(img.shape)==3:
        return process(img[:,:,0],kernel)
    else:
        return process(img,kernel)

#Definicion de la funcion Bartlett de 7x7
def bart_7x7(img):
    vec= [1,2,3,4,3,2,1]
    [x,y] = np.meshgrid(vec,vec)
    kernel = (x*y)/np.sum(x*y)
    if len(img.shape)==3:
        return process(img[:,:,0],kernel)
    else:
        return process(img,kernel)

#Definicion de la funcion de Gauss de 5x5
def gauss_5x5(img):
    vec= [1,4,6,4,1]
    [x,y] = np.meshgrid(vec,vec)
    kernel = (x*y)/np.sum(x*y)
    if len(img.shape)==3:
        return process(img[:,:,0],kernel)
    else:
        return process(img,kernel)

#Definicion de la funcion de Gauss de 7x7
def gauss_7x7(img):
    vec= [1,6,15,20,15,6,1]
    [x,y] = np.meshgrid(vec,vec)
    kernel = (x*y)/np.sum(x*y)
    if len(img.shape)==3:
        return process(img[:,:,0],kernel)
    else:
        return process(img,kernel)

#Definicion de la funcion Laplaciana de 3x3 con 4 vecinos
def lapla_v4(img): 
    kernel = np.array([[0,-1,0],[-1,4,-1],[0,-1,0]])   #Se define el kernel
    if len(img.shape)==3:
        return process(img[:,:,0],kernel)
    else:
        return process(img, kernel)

#Definicion de la funcion Laplaciana de 3x3 con 8 vecinos
def lapla_v8(img): 
    kernel = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]])   #Se define el kernel
    return process(img,kernel)

#Funcion Sobel, el cual tambien incluye como parametro un Combobox el cual contendra la direccion seleccionada por el usuario
def sobel(img,combo):
    if combo.get() == 'N':                                     #Definicion de los kernels para cada orientacion
        kernel = np.array([[-1,-2,-1],[0,0,0],[1,2,1]])
    elif combo.get() == 'S':
        kernel = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
    elif combo.get() == 'E':
        kernel = np.array([[1,0,-1],[2,0,-2],[1,0,-1]])
    elif combo.get() == 'O':
        kernel = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
    elif combo.get() == 'NE':
        kernel = np.array([[0,-1,-2],[1,0,-1],[2,1,0]])
    elif combo.get() == 'NO':
        kernel = np.array([[-2,-1,0],[-1,0,1],[0,1,2]])
    elif combo.get() == 'SE':
        kernel = np.array([[2,1,0],[1,0,-1],[0,-1,-2]])
    else:
        kernel = np.array([[0,1,2],[-1,0,1],[-2,-1,0]])
    if len(img.shape)==3:
        return process(img[:,:,0],kernel)
    else:
        return process(img,kernel)

In [3]:
window = Tk()

window.title("Trabajos")
window.geometry('1100x950')
tab_control = ttk.Notebook(window)
tab1 = ttk.Frame(tab_control)
tab2 = ttk.Frame(tab_control)
tab3 = ttk.Frame(tab_control)
tab_control.add(tab1, text='Tp 4')
tab_control.add(tab2, text='Tp 5')
tab_control.add(tab3, text='Tp 6')

#Configuracion de Ventana del Trabajo Practico 4
#Definicion de la funcion click del boton "Procesar Imagen", el cual invocara la funcion correspondiente a la seleccion del usuario
def clicked4(image,combo,combo2):
    if combo.get() == 'Plano':
        res = plano(image)
    elif combo.get() == 'Bartlett 3x3':
        res = bart_3x3(image)
    elif combo.get() == 'Bartlett 5x5':
        res = bart_5x5(image)
    elif combo.get() == 'Bartlett 7x7':
        res = bart_7x7(image)
    elif combo.get() == 'Gaussiano 5x5':
        res = gauss_5x5(image)
    elif combo.get() == 'Gaussiano 7x7':
        res = gauss_7x7(image)
    elif combo.get() == 'Laplaciano v4':
        res = lapla_v4(image)
    elif combo.get() == 'Laplaciano v8':
        res = lapla_v8(image)
    else:
        res = sobel(image,combo2)
    
    #La figura resulatnte se muestra en la ventana
    fig4_1 = plt.figure(figsize=(5,5))
    plt.axis('off')
    plt.imshow(res) 
    canvas = FigureCanvasTkAgg(fig4_1, master = tab1)
    canvas.draw()
    canvas.get_tk_widget().grid(padx=15,pady=5,column=2,row=1,columnspan = 2)
    
#Definicion de la funcion click del boton "Abrir imagen". Carga la imagen y la muestra en la ventana       
def clickedopen4():
    image4 = np.clip(imageio.imread(filedialog.askopenfilename())/255.,0.,1.)
    fig4 = plt.figure(figsize=(5,5))
    plt.axis('off')
    plt.imshow(image4) 
    canvas = FigureCanvasTkAgg(fig4, master = tab1)
    canvas.draw()
    canvas.get_tk_widget().grid(padx=15,pady=15,column=0,row=1,columnspan=2)
    
    lbl4_3 = Label(tab1, text= 'Seleccione operacion:',font=("Arial Bold",12))
    lbl4_3.grid(column=0, row=3)

    #Se crea un combobox para la seleccion del operador
    combo4 = ttk.Combobox(tab1, state="readonly")
    combo4['values'] = ('Plano','Bartlett 3x3','Bartlett 5x5', 'Bartlett 7x7', 'Gaussiano 5x5', 'Gaussiano 7x7', 'Laplaciano v4','Laplaciano v8','Sobel')
    combo4.current(0)
    combo4.grid(padx = 15, pady = 15, column = 1, row = 3)
        
    lbl4_4 = Label(tab1, text= 'Seleccione orientacion (Sobel):',font=("Arial Bold",12))
    lbl4_4.grid(column=0, row=4)

    #Se crea un combobox para la seleccion de la direccion, cuando se opera con el operador Sobel
    combo4_1 = ttk.Combobox(tab1, state="readonly")
    combo4_1['values'] = ('E','O','N', 'S', 'NE', 'NO', 'SE','SO')
    combo4_1.current(0)
    combo4_1.grid(padx = 15, pady = 15, column = 1, row = 4)
    
    btn4 = Button(tab1, text="Procesar Imagen", width=50, command=lambda: clicked4(image4,combo4, combo4_1))                               #state='disabled'
    btn4.grid(pady=15,column=0,row=5,columnspan=2)
        

lbl4_1 = Label(tab1, text= 'Imagen Original',font=("Arial Bold",18))
lbl4_1.grid(column=0, row=0, columnspan=2)

lbl4_2 = Label(tab1, text= 'Imagen Modificada', font=("Arial Bold",18),width=30)
lbl4_2.grid(column=2, row=0) 

btnopen4 = Button(tab1, text="Abrir Imagen", width=50, command=lambda: clickedopen4())                               #state='disabled'
btnopen4.grid(pady=15,column=0,row=2,columnspan=2)



In [ ]:

tab_control.pack(expand=1, fill='both')
window.mainloop()

